In [ ]:
"""Time of execution for Colab free."""

**Imports.**

**Data parsing.**

**Preprocess.**

    Data cleaning.
    Tokenization.
    Vectorizing.
    
**Cosim similarity (Clastering).**
    
**Machine Learning.**

**Bonus. Word2Vec.**


**Imports:**

In [ ]:
"""Check environment"""
!pwd
!which python

In [ ]:
"""Install & import libraries for NLP and more"""

#! pip install -U textblob
#! pip install --upgrade scikit-learn
#! pip install nltk
#! pip install gensim
#! pip install tensorflow

"""Install the package is capable of resolving contractions (and slang): contractions"""
#! pip install contractions

"""Install Open source library for Emoticons and Emoji detection library: emot"""
#! pip install emot

import nltk
nltk.download("popular")
import pandas as pd
import numpy as np
import re
import string


In [ ]:
"""Deeplearning library import, spetial solution"""

import tensorflow
# TF_ENABLE_ONEDNN_OPTS=0

In [ ]:
####################################################################################################

**Data parsing:**

In [ ]:
"""Function for read files, marking(0 or 1) and join dataframes then."""

def get_data():
    df_pos = pd.read_csv('./data/processedPositive.csv').transpose().reset_index()
    df_pos['tone'] = 1
    df_neg = pd.read_csv('./data/processedNegative.csv').transpose().reset_index()
    df_neg['tone'] = -1
    df_neu = pd.read_csv('./data/processedNeutral.csv').transpose().reset_index()
    df_neu['tone'] = 0
    df_data = pd.concat([df_pos, df_neg, df_neu])
    df_data_save = df_data.copy()
    return df_data, df_data_save

In [ ]:
df_data, df_data_save = get_data()
print(df_data.head())
print(df_data.describe(), '\n')
print(type(df_data))

# display(df_data_save[16:3000:500])
# print(df_data_save.describe(), '\n')
# print(type(df_data_save))

In [ ]:
####################################################################################################

**Preprocess:**

**Data cleaning.**

    Contraction to full form
    Replace emoticons with text

    Remove: 
          ticks and next symbol
          url (http*)
          hashtags (#)
          mentions (@)
          stopwords
          extra spaces
          punctuation

    Drop duplicates

In [ ]:
#   !! WALKING AROUND !!
"""Find all mentions in Dataframe"""

text = "Beautiful album greatest unsung guitar genius @PeterPen @BobbySquash"
list_of_mentions = []
for txt in df_data['index']:
    mentions = (re.findall("@[A-Za-z0-9_]{1,50}", txt))
    if mentions:
        list_of_mentions+=(mentions)
mentions = (re.findall("@[A-Za-z0-9_]{1,50}", text))
if mentions:
      list_of_mentions+=(mentions)
    # list_of_mentions+=re.findall("@[A-Za-z0-9_]{1,50}", text)
print('1) list_of_mentions:',list_of_mentions)

#   Find all hashtags in Dataframe
list_of_hashtags = []
text = "Beautiful album greatest unsung guitar genius #KingPing #DingDongKong"
for txt in df_data['index']:
    hashtags = (re.findall("#[A-Za-z0-9_]{1,50}", txt))
    if hashtags:
        list_of_hashtags+=(hashtags)
hashtags = (re.findall("#[A-Za-z0-9_]{1,50}", text))
if hashtags:
      list_of_hashtags+=(hashtags)
    # list_of_hashtags+=re.findall("#[A-Za-z0-9_]{1,50}", text)
print('2) list_of_hashtags:', list_of_hashtags)

#   !! WALKING AROUND !!
"""Example is cleaning Dataframe from URLs with re.exp."""
text = "This text has or had URL https://profile.intra.42.fr/# inside!"
clear = re.sub('http\S+','',text)
print('1) clear:', clear)

"""Example is cleaning Dataframe from URLs with urlparse and list compehension"""
from urllib.parse import urlparse
text = "This text has or had URLs https://profile.intra.42.fr/# and https://21-school.ru/ inside!"
not_url_list = [item for item in text.split() if not urlparse(item).scheme]
clear = ' '.join(not_url_list)
print('2) clear:', clear)

#   !! WALKING AROUND !!
"""Find URLs in Dataframe"""
def find_url(text, url_lst):
    urls = (re.findall('http\S+', text))
    if urls:
        url_lst = url_lst+urls
    return url_lst

url_lst = []
url_lst = df_data['index'].apply(lambda x: find_url(x, url_lst))
for url in url_lst:
      if url:
        print(url)

In [ ]:
df_data, df_data_save = get_data()      # get data and make copy of DataFrame

####################################################################################################
"""Function fix contractions"""

import contractions
df_data['index'] = df_data['index'].apply(lambda x: contractions.fix(x))

####################################################################################################
"""Function replace emoticons with text"""

def emoticon_transform(text, emot):
    emoticons = emot.emoticons(text).get('value')
    means = emot.emoticons(text).get('mean')
    if emoticons:
        for i, emot in enumerate(emoticons):
            text = text.replace(emot, ' ' + means[i] + ' ')
    return text

import emot

emot = emot.core.emot()
df_data['index'] = df_data['index'].apply(lambda x: emoticon_transform(x, emot))

####################################################################################################
"""Function is cleaning Dataframe from english stopwords"""

def drop_stopwords(text, en_stopwords):
    # print(en_stopwords)
    tokens_lst = []
    for token in text.split():
        if token.lower() not in en_stopwords:
            tokens_lst.append(token)

    return ' '.join(tokens_lst)

from nltk.corpus import stopwords

en_stopwords = stopwords.words('english')
df_data['index'] = df_data['index'].apply(lambda x: drop_stopwords(x, en_stopwords))

####################################################################################################
"""Function is cleaning Dataframe from hashtags and mentions"""

def drop_hashtags_and_mentions(text):
    clean_text = re.sub("@[A-Za-z0-9_]+","",text) # replace mention with empty string
    clean_text = re.sub("#[a-zA-Z0-9_]+","",clean_text) # replace hashtag with empty string
    return clean_text

df_data['index'] = df_data['index'].apply(lambda x: drop_hashtags_and_mentions(x))

####################################################################################################
"""Function is cleaning Dataframe from URLs"""

def drop_url(text):
    return re.sub('http\S+','',text)

df_data['index'] = df_data['index'].apply(lambda x: drop_url(x))

####################################################################################################
"""Function is cleaning Dataframe from ticks 
    and the next character (do it when contractions has fixed only)"""

def drop_ticks_and_nextone(text):
    return re.sub(r"\'\w+",'',text)

df_data['index'] = df_data['index'].apply(lambda x: drop_ticks_and_nextone(x))

####################################################################################################
"""Function is cleaning Dataframe from numbers"""

def drop_numbers(text):
    return re.sub(r"\d+", ' ',text)

df_data['index'] = df_data['index'].apply(lambda x: drop_numbers(x))

####################################################################################################
"""Function is cleaning Dataframe from punctuations"""

def drop_punctuations(text):
    return re.sub('[%s]' % re.escape(string.punctuation), ' ', text)

df_data['index'] = df_data['index'].apply(lambda x: drop_punctuations(x))

####################################################################################################
"""Function is converting Dataframe to lower case"""

df_data['index'] = df_data['index'].apply(lambda x: x.lower())

####################################################################################################
"""Function is cleaning Dataframe from numbers"""

def drop_over_spaces(text):
    text = ' '.join(text.split())
    return re.sub(r"\s{2,}", ' ',text)

df_data['index'] = df_data['index'].apply(lambda x: drop_over_spaces(x))

####################################################################################################
"""Function is makeing Dataframe with unique rows"""

df_data = df_data.drop_duplicates(subset='index')

####################################################################################################
"""Function is droping empty rows"""

df_data = df_data.replace(r'', float('NaN'), regex=True).dropna()

print(df_data[25:3025:525])

In [ ]:
####################################################################################################

**Tokenization.**

    Just tokenization
    Stemming (2 ways)
    Lematization
    Stemming + misspelling
    Lematization + misspelling

In [ ]:
####################################################################################################
"""Just tokenize (by sentence) - deiding text to text units"""

def tokenization(text):
    from nltk.tokenize import word_tokenize
    return (word_tokenize(text))

df_tokenized = df_data.copy()
df_tokenized['processed'] = df_tokenized['index'].apply(lambda x: tokenization(x))
display('1. df_tokenized:', df_tokenized.head())

####################################################################################################
"""Stemming (by word) - removing word endings (PorterStemmer)"""

def stemming(text, stemmer):
    text = text.split()
    stem_lst = list()
    for word in text:
        stem = stemmer.stem(word)
        stem_lst.append(stem)
    return (' '.join(stem_lst))

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
df_stemmed = df_data.copy()
df_stemmed['processed'] = (df_stemmed['index']
                                            .apply(lambda x: stemming(x, stemmer))
                                            .apply(lambda x: tokenization(x))) 
display(' --------------------------------------------- ','2. df_stemmed:', df_stemmed.head())

####################################################################################################
"""Stemming (by word) - removing word endings (LancasterStemmer)"""

from nltk.stem import LancasterStemmer
stemmer_lc = LancasterStemmer()
df_stemmed_lc = df_data.copy()
df_stemmed_lc['processed'] = (df_stemmed_lc['index']
                                            .apply(lambda x: stemming(x, stemmer_lc))
                                            .apply(lambda x: tokenization(x)))
display(' --------------------------------------------- ','3. df_stemmed_lc:',df_stemmed_lc.head())

####################################################################################################
"""Lemmatization (by word) - converting to normal form"""

def lemmatization(text, lemmatizer):
    text = text.split()
    lemm_lst = list()
    for word in text:
        lemm = lemmatizer.lemmatize(word)
        lemm_lst.append(lemm)
    return (' '.join(lemm_lst))

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
df_lemmatized = df_data.copy()
df_lemmatized['processed'] = (df_lemmatized['index']
                                            .apply(lambda x: lemmatization(x, lemmatizer))
                                            .apply(lambda x: tokenization(x))) 
display(' --------------------------------------------- ','4. df_lemmatized:', df_lemmatized.head())


In [ ]:
####################################################################################################
"""Fix misspelings with TextBlob (~7 minutes)"""

def correct_spelling(text):
    from textblob import TextBlob
    text = TextBlob(text)
    return str(text.correct())

df_misspelling_off = df_data.copy()
df_misspelling_off['processed'] = df_misspelling_off['index'].apply(lambda x: correct_spelling(x))

####################################################################################################
"""Stemming (by word) - removing word endings (PorterStemmer) + df_misspelling_off"""

stemmer = PorterStemmer()

df_stemmed_plus = df_misspelling_off.copy()
df_stemmed_plus['processed'] = (df_stemmed['index']
                                            .apply(lambda x: stemming(x, stemmer))
                                            .apply(lambda x: tokenization(x))) 
display(' --------------------------------------------- ','5. df_stemmed_plus :',df_stemmed_plus)

####################################################################################################
"""Stemming (by word) - removing word endings (LancasterStemmer) + df_misspelling_off"""

stemmer_lc = LancasterStemmer()

df_stemmed_lc_plus = df_data.copy()
df_stemmed_lc_plus['processed'] = (df_stemmed['index']
                                            .apply(lambda x: stemming(x, stemmer))
                                            .apply(lambda x: tokenization(x)))
display(' --------------------------------------------- ','6. df_stemmed_lc_plus:',df_stemmed_lc_plus.head())

####################################################################################################
"""Lemmatization (by word) + df_misspelling_off"""

lemmatizer = WordNetLemmatizer()

df_lemmatized_plus = df_misspelling_off.copy() 
df_lemmatized_plus['processed'] = (df_lemmatized['index']
                                            .apply(lambda x: lemmatization(x, lemmatizer))
                                            .apply(lambda x: tokenization(x))) 
display(' --------------------------------------------- ','7. df_lemmatized_plus :',df_lemmatized_plus)

In [ ]:
####################################################################################################

**Vectorizing**:

    Word embedding:
        One-Hot-Encoding
        Bag of words
        TF-IDF
    
        Word2vec.




In [ ]:
##   !! WALKING AROUND !!
"""One-Hot-Encoding (SLOW experiment)"""

one_hot_encoding = pd.DataFrame(index=corpus)
for document in corpus:
    for token in document.split():
        if token not in one_hot_encoding.columns:
            one_hot_encoding[token] = 0

def mark(row):
    for word, mark in row.iteritems():
        if word in row.name.split():
            row[word] = 1
    return row

one_hot_encoding = one_hot_encoding.apply(lambda row: mark(row), axis=1)

In [ ]:
one_hot_encoding

In [ ]:
"""Some ways to vectorize"""

def vectorize(df_for_vectorizing):
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.feature_extraction.text import TfidfVectorizer 
    
    corpus = df_for_vectorizing['processed'].apply(lambda row: ' '.join(row))
    index_s = df_for_vectorizing['index']

    """Word count: Bag-of-Words 
    — это статистический анализ, анализирующий количественное вхождение слов в документах."""
    bow_vectorizer = CountVectorizer()
    bow_matrix = bow_vectorizer.fit_transform(corpus).toarray()
    bag_of_words = pd.DataFrame(data=bow_matrix, 
                                index=index_s, 
                                columns=bow_vectorizer.get_feature_names_out())
    """One-Hot-Encoding (0 or 1, if the word exist)
    - быстрая разметка"""
    one_hot_encoding = bag_of_words.copy()
    for column in one_hot_encoding.columns:
        one_hot_encoding[column] = (one_hot_encoding[column]
                                    .apply(lambda item: item if not item else 1))

    """TF-IDF 
    (от англ. TF — term frequency, IDF — inverse document frequency) — статистическая мера,
    используемая для оценки важности слова в контексте документа, являющегося частью коллекции документов или корпуса.
    Вес некоторого слова пропорционален частоте употребления этого слова в документе
    и обратно пропорционален частоте употребления слова во всех документах коллекции."""

    tf_idf_vectorizer = TfidfVectorizer()
    tf_idf_matrix = tf_idf_vectorizer.fit_transform(corpus).todense()
    tf_idf = pd.DataFrame(data=tf_idf_matrix, 
                        index = index_s,
                        columns = tf_idf_vectorizer.get_feature_names_out())
    
    return one_hot_encoding, bag_of_words, tf_idf

one_hot_encoding, bag_of_words, tf_idf = vectorize(df_tokenized)


In [ ]:
one_hot_encoding[one_hot_encoding['zoo']>0]

In [ ]:
####################################################################################################

**Cosim similarity (Clastering):**

In [ ]:
"""Find 10 most similar tweets one for each (~12 min)
    return TOP 10 for each"""

top_counter = 0     # Count result tables

tokenizers = { 
                'just_tokenization':df_tokenized,
                'stemming Porter': df_stemmed,
                'lematization': df_lemmatized,
                'stemming Lancaster': df_stemmed_lc,
                'stemming Porter + misspelling': df_stemmed_plus,
                'stemming Lancaster + misspelling': df_stemmed_lc_plus,
                'lematization + misspelling': df_lemmatized_plus,

                }

for tokenizer_key in tokenizers:
    print('TOKENIZER -', tokenizer_key)
    df_tokens = tokenizers.get(tokenizer_key)
    one_hot_encoding, bag_of_words, tf_idf = vectorize(df_tokens) # here we make vectors
    vectorizers = {
                    '0 or 1, if the word exist':one_hot_encoding,
                    'word count':bag_of_words,
                    'tf-idf':tf_idf
                    }
    for vectoriser_key in vectorizers:
        print('#', top_counter)
        print('Vectorizer -', vectoriser_key)
        df_vectors = vectorizers.get(vectoriser_key)

        """Making matrix of cosine_semilarity for dataset's vectors"""
        from scipy.spatial import distance
        from sklearn.metrics.pairwise import cosine_similarity

        cos_sim = cosine_similarity(df_vectors.values) # here we make calculation
        df_similarity = pd.DataFrame(cos_sim, columns=df_vectors.index.values, index=df_vectors.index) # here we make matrix

        """Make TOP-10 similar tweets DataFrame"""
        top = pd.DataFrame(columns = ['tweet_x','tweet_y','cosine_similarity'])

        def new_top_candidat(X,Y,rate):
            """function make new row for appending to top dataframe"""
            rate_data = {'tweet_x':[X],'tweet_y':[Y],'cosine_similarity':[rate]}
            df_top_candidat = pd.DataFrame(rate_data)
            return df_top_candidat

        for Y, row in df_similarity.iterrows():
            if Y == 'omg he kissed crying joy':                                                     # this is key-tweet
                for X, rate in row.iteritems():
                    if X != Y and rate > 0 and rate < 1:                                            # rules for filtering      
                        if top.shape[0] < 10:
                            df_top_candidat = new_top_candidat(X,Y,rate) 
                            top = (top.append(df_top_candidat)
                                    .sort_values(by='cosine_similarity', ascending=False))
                            del(df_top_candidat)
                        else:
                            min_rate = top.cosine_similarity.min()
                            if min_rate < rate:
                                top = top.head(9)                                                   # get just top-9
                                df_top_candidat = new_top_candidat(X,Y,rate)                        # make 10th for adding
                                top = (top.append(df_top_candidat)
                                        .sort_values(by='cosine_similarity', ascending=False)       # sort descending by one column
                                        .drop_duplicates(subset='cosine_similarity'))               # drop duplicates by one column
                                del(df_top_candidat)

        if top.shape[0] == 10:
            top = top.set_index(pd.Index(range(1,11)))
        top_counter+=1
        display(top)
        # print('Tweet X:\n',top.tweet_x.values, '\n')
        # print('Tweet Y:\n',top.tweet_y.values)
display('TOTAL TABLES =', top_counter)
top.cosine_similarity.describe()


In [ ]:
##   !! WALKING AROUND !!

one_hot_encoding, bag_of_words, tf_idf = vectorize(df_tokens)                                       # here we make vector
df_similarity = pd.DataFrame(cos_sim, columns=tf_idf.index.values, index=tf_idf.index)              # here we make matrix
for Y, row in df_similarity.iterrows():                                                             
    print(Y)                                                                                        # here we print all tweets
    

In [ ]:
##   !! WALKING AROUND !!
"""Making TOP with different tweets"""

top = pd.DataFrame(columns = ['tweet_x','tweet_y','cosine_similarity'])

def new_top_candidat(X,Y,rate):
    """function make new row for appending to top dataframe"""
    rate_data = {'tweet_x':[X],'tweet_y':[Y],'cosine_similarity':[rate]}
    df_top_candidat = pd.DataFrame(rate_data)
    return df_top_candidat

for Y, row in df_similarity.iterrows():
    """Uncomment text or write some new"""
    if Y == 'indian heft pope':
    # if Y == 'omg he kissed crying joy':
    # if Y == 'britain becoming xenophobic hostile':
    # if Y == 'supreme court restores bangalore trial court verdict holding others corruption case':
        for X, rate in row.iteritems():
            if X != Y and rate > 0 and rate < 1: # Rules for filtering        
                if top.shape[0] < 10:
                    df_top_candidat = new_top_candidat(X,Y,rate) 
                    top = (top.append(df_top_candidat)
                            .sort_values(by='cosine_similarity', ascending=False))
                    del(df_top_candidat)
                else:
                    min_rate = top.cosine_similarity.min()
                    if min_rate < rate:
                        top = top.head(9)
                        df_top_candidat = new_top_candidat(X,Y,rate)
                        top = (top.append(df_top_candidat)
                                .sort_values(by='cosine_similarity', ascending=False) # sort descending by one column
                                .drop_duplicates(subset='cosine_similarity')) # drop duplicates by one column
                        del(df_top_candidat)
if top.shape[0] == 10:
    top = top.set_index(pd.Index(range(1,11)))
    
display(top)


In [ ]:
##   !! WALKING AROUND !!
"""Find 10 most similar tweets all for each (not very interesting, but maximum similarity rates)"""

tokenizers = { 
                # 'just_tokenization':df_tokenized,
                # 'stemming Porter': df_stemmed,
                # 'lematization': df_lemmatized,
                # 'stemming Lancaster': df_stemmed_lc,
                # 'stemming Porter + misspelling': df_stemmed_plus,
                'lematization + misspelling': df_lemmatized_plus,
                # 'stemming Lancaster + misspelling': df_stemmed_plus,
                }

for tokenizer_key in tokenizers:
    print('TOKENIZER -', tokenizer_key)
    df_tokens = tokenizers.get(tokenizer_key)
    one_hot_encoding, bag_of_words, tf_idf = vectorize(df_tokens) # here we make vectors
    vectorizers = {
                    '0 or 1, if the word exist':one_hot_encoding,
                    'word count':bag_of_words,
                    'tf-idf':tf_idf
                    }
    for vectoriser_key in vectorizers:
        print('Vectorizer -', vectoriser_key)
        df_vectors = vectorizers.get(vectoriser_key)

        """Making matrix of cosine_semilarity for dataset's vectors"""
        from scipy.spatial import distance
        from sklearn.metrics.pairwise import cosine_similarity

        cos_sim = cosine_similarity(df_vectors.values) # here we make calculation
        df_similarity = pd.DataFrame(cos_sim, columns=df_vectors.index.values, index=df_vectors.index) # here we make matrix

        """Make TOP-10 similar tweets DataFrame"""
        top = pd.DataFrame(columns = ['tweet_x','tweet_y','cosine_similarity'])

        def new_top_candidat(X,Y,rate):
            """function make new row for appending to top dataframe"""
            rate_data = {'tweet_x':[X],'tweet_y':[Y],'cosine_similarity':[rate]}
            df_top_candidat = pd.DataFrame(rate_data)
            return df_top_candidat

        for Y, row in df_similarity.iterrows():
            for X, rate in row.iteritems():
                if X != Y and rate > 0 and rate < 1: # Rules for filtering        
                    if top.shape[0] < 10:
                        df_top_candidat = new_top_candidat(X,Y,rate) 
                        top = (top.append(df_top_candidat)
                                .sort_values(by='cosine_similarity', ascending=False))
                        del(df_top_candidat)
                    else:
                        min_rate = top.cosine_similarity.min()
                        if min_rate < rate:
                            top = top.head(9)
                            df_top_candidat = new_top_candidat(X,Y,rate)
                            top = (top.append(df_top_candidat)
                                    .sort_values(by='cosine_similarity', ascending=False) # sort descending by one column
                                    .drop_duplicates(subset='cosine_similarity')) # drop duplicates by one column
                            del(df_top_candidat)

        top = top.set_index(pd.Index(range(1,11)))
        display(top)
        # print('Tweet X:\n',top.tweet_x.values, '\n')
        # print('Tweet Y:\n',top.tweet_y.values)
top.cosine_similarity.describe()


In [ ]:
####################################################################################################

**Machine Learning:**

In [ ]:
##   !! WALKING AROUND !!
"""Creating DataFrame and adding one row with custom index"""

df_res = pd.DataFrame(columns = ['0 or 1, if the word exist', 'word_count', 'TFIDF'])
df_res = df_result.append(pd.DataFrame(data={ '0 or 1, if the word exist':[2222], 'word_count':[1111], 'TFIDF':[3333]}, index=['way']))
df_res

In [ ]:
"""Learning some ML moodels (~42 min)"""

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.svm import SVC

tokenizers = { 
                'just_tokenization':df_tokenized,
                'stemming Porter': df_stemmed,
                'stemming Lancaster': df_stemmed_lc,
                'lematization': df_lemmatized,
                'stemming Porter + misspelling': df_stemmed_plus,
                'stemming Lancaster + misspelling': df_stemmed_plus,
                'lematization + misspelling': df_lemmatized_plus,
             }

df_result = pd.DataFrame(columns = ['0 or 1, if the word exist', 'word_count', 'TFIDF'])            # create results table
for tokenizer_key in tokenizers:
    print('\n', 'TOKENIZER -', tokenizer_key)
    df_tokens = tokenizers.get(tokenizer_key)                                                       # get next processor
    one_hot_encoding, bag_of_words, tf_idf = vectorize(df_tokens)                                   # get vectors for processed data
    vectorizers = { 
                    'One-Hot-Encoding':one_hot_encoding,
                    'Bag Of Words':bag_of_words,
                    'TF-IDF':tf_idf,
                  }
    result_list = list()
    for vectoriser_key in vectorizers:
        scores = pd.Series([], dtype='object')
        df_vector = vectorizers.get(vectoriser_key)
        X_train, X_test, y_train, y_test = train_test_split(df_vector, df_data.tone, test_size=0.2, random_state=42)
        print('\n', 'Preprocessing way:', vectoriser_key)
        """X - preprocessed data, y - data marks"""
        print('Train dataset part size:',X_train.shape, y_train.shape) 
        print('Test dataset part size:',X_test.shape, y_test.shape)
        print('============')

        """start learning models"""

        """Гауссовский наивный байесовский реализует гауссовский наивный байесовский алгоритм для классификации. 
        Предполагается, что вероятность появления признаков гауссова"""
        clf = GaussianNB()         
        clf.fit(X_train, y_train)
        scores = scores.append(pd.Series([clf.score(X_test, y_test)]))
        print('Gausian Native Bayes accuracy -',clf.score(X_test, y_test))

        """Логистическая регрессия — статистическая модель, используемая для прогнозирования 
        вероятности возникновения некоторого события путём его сравнения с логистической кривой. 
        Эта регрессия выдаёт ответ в виде вероятности бинарного события (1 или 0)."""
        clf = LogisticRegression(random_state=42)
        clf.fit(X_train, y_train)
        scores = scores.append(pd.Series([clf.score(X_test, y_test)]))
        print('Logistic Regression accuracy -',clf.score(X_test, y_test))

        """Random Forest — это множество решающих деревьев. 
        В задаче регрессии их ответы усредняются,
        в задаче классификации принимается решение голосованием по большинству."""
        clf = RandomForestClassifier(max_depth=2, random_state=42)
        clf.fit(X_train, y_train)
        scores = scores.append(pd.Series([clf.score(X_test, y_test)]))
        print('Random Forest accuracy -',clf.score(X_test, y_test))

        clf = RandomForestClassifier(n_estimators = 1025, criterion='entropy', random_state=42)
        clf.fit(X_train, y_train)
        scores.append(pd.Series([clf.score(X_test, y_test)]))
        print('Random Forest with entropy accuracy -',clf.score(X_test, y_test))

        """Деревья решений (DT) — это непараметрический контролируемый метод обучения, 
        используемый для классификации и регрессии. 
        Цель состоит в том, чтобы создать модель, которая предсказывает значение целевой переменной, 
        изучая простые правила принятия решений, выведенные из характеристик данных. 
        Дерево можно рассматривать как кусочно-постоянное приближение."""
        clf = tree.DecisionTreeClassifier()
        clf.fit(X_train, y_train)
        scores.append(pd.Series([clf.score(X_test, y_test)]))
        print('Decision Tree accuracy -',clf.score(X_test, y_test))

        """Метод опорных векторов (англ. SVM, support vector machine) — набор схожих алгоритмов обучения с учителем, 
        использующихся для задач классификации и регрессионного анализа. 
        Принадлежит семейству линейных классификаторов и может также рассматриваться как частный случай регуляризации по Тихонову. 
        Особым свойством метода опорных векторов является непрерывное уменьшение эмпирической ошибки классификации и увеличение зазора, 
        поэтому метод также известен как метод классификатора с максимальным зазором."""
        clf = SVC(kernel='linear', C=1)
        clf.fit(X_train, y_train)
        scores.append(pd.Series([clf.score(X_test, y_test),]))
        print('C-Support Vector Classification accuracy -',clf.score(X_test, y_test))

        result_list.append(scores.max())                                                            # results for different BOW to one tokenizer
    df_result = (df_result.append(
                                  pd.DataFrame(np.array([result_list]), 
                                               columns=['0 or 1, if the word exist', 'word_count', 'TFIDF'], 
                                               index=[tokenizer_key]))
                                 )                                                                  # add result row 

                                                                                  

"""Do gridsearch to find the best parameters for ML algorithm (SVC)"""

def do_gridsearch(alg, X_train, y_train):
    from sklearn.model_selection import GridSearchCV
    parameters = [{
            'kernel': ['linear', 'poly', 'rbf'],
            'C': [1,25],
            'max_iter': [10,100]
            }]

    clf = GridSearchCV(alg, parameters, scoring='accuracy')
    clf.fit(X_train, y_train)
    print('GridSearch accuracy params -',clf.best_params_)
    print('-------------')

alg = SVC()
do_gridsearch(alg, X_train, y_train)                                                                # start gridsearch

display(df_result)                                                                                  # results of ML accuracy


In [ ]:
##   !! WALKING AROUND !!

from sklearn.svm import SVC
def do_gridsearch(alg, X_train, y_train):
    from sklearn.model_selection import GridSearchCV
    parameters = [{
            'kernel': ['linear', 'poly', 'rbf'],
            'C': [1,125],
            'max_iter': [10,100]
            }]

    clf = GridSearchCV(alg, parameters, scoring='accuracy')
    clf.fit(X_train, y_train)
    print('GridSearch accuracy params -',clf.best_params_)
    print('-------------')

alg = SVC()
do_gridsearch(alg, X_train, y_train)                                                                     # start gridsearch


In [ ]:
####################################################################################################

**Bonus. Word2Vec.**

In [ ]:
"""! GENSIM 4.0 !"""

corpus = df_data['index'] #df_tokenized_plus['processed'].apply(lambda row: ' '.join(row))

from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import text_to_word_sequence

"""Vectorizing with Keras libruary"""

tweets_lists = [text_to_word_sequence(tweet) for tweet in corpus]
tweets_lists = tweets_lists


"""Initialazing Word2Vec model"""
w2v_model = Word2Vec(sentences=tweets_lists, window=5, min_count=1)

"""First training Word2Vec model"""
w2v_model.train(tweets_lists, total_examples=len(tweets_lists), epochs=8)


In [ ]:
"""Some useful from Word2Vec"""

all_words = list(w2v_model.wv.index_to_key)
print('\n', all_words,'\n')

# display('Word vector:', w2v_model.wv.word_vec('happy'))
display('Word vector:', w2v_model.wv.get_vector('happy'))
print('\n')

WORD = 'happy'
display(f"Most similar words to word '{WORD}':", w2v_model.wv.most_similar(WORD))

In [13]:
"""Machine learning with Word2Vec"""

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

"""Making embeddings"""
def get_embeddings(tokens,vector):
    def get_word_embeddings(token_word_list,vector,k=100):
        if len(token_word_list) < 1:
            return np.zeros(k)
        else:
            vectorized = ([vector.wv.get_vector(word) 
                           if word in vector.wv.index_to_key 
                           else np.random.rand(k) 
                           for word in token_word_list]) 

        summ = np.sum(vectorized,axis=0)
        ## return the average
        return summ/len(vectorized)

    display(tokens)
    embeddings = tokens.apply(lambda x: get_word_embeddings(x, vector))
    return list(embeddings)

embeddings = get_embeddings(pd.Series(tweets_lists),w2v_model)


"""X - preprocessed data, y - data marks"""
X_train, X_test, y_train, y_test = train_test_split(embeddings, df_data.tone, test_size=0.2, random_state=42) 
# print('\n', 'Train dataset part:',X_train[:2], y_train[:2]) 
# print('\n', 'Test dataset part:',X_test[:2], y_test[:2])
# print('||||||||||||')

"""start learning models"""

word2vec = LogisticRegression(solver='lbfgs', max_iter=42)
word2vec.fit(X_train, y_train)
print('\n', 'Logistic Regression accuracy with Word2Vec -', word2vec.score(X_test, y_test), '\n')


NameError: name 'pd' is not defined

In [ ]:
"""More training Word2Vec model (~1 min).
    Need just one more to get 15% better acuracy."""

w2v_model.train(tweets_lists, total_examples=len(tweets_lists), epochs=125)

In [9]:
!git add *

In [10]:
!git status


On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   data/processedNegative.csv
	new file:   data/processedNeutral.csv
	new file:   data/processedPositive.csv
	new file:   tweets.ipynb



In [11]:
!git commit -m "Ready to get better"

[main b3739f1] Ready to get better
 4 files changed, 1650 insertions(+)
 create mode 100644 data/processedNegative.csv
 create mode 100644 data/processedNeutral.csv
 create mode 100644 data/processedPositive.csv
 create mode 100644 tweets.ipynb


In [12]:
!git push

Enumerating objects: 8, done.
Counting objects: 100% (8/8), done.
Delta compression using up to 5 threads
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 84.98 KiB | 3.15 MiB/s, done.
Total 7 (delta 0), reused 0 (delta 0)
To github.com:Dr-Code-lab/NLP_analyze.git
   3a1d8f1..b3739f1  main -> main
